In [1]:
import sys
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles', '/home/vinnylg/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles/lib/python', '/home/vinnylg/anaconda3/envs/boletim/lib/python38.zip', '/home/vinnylg/anaconda3/envs/boletim/lib/python3.8', '/home/vinnylg/anaconda3/envs/boletim/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/envs/boletim/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
import numpy as np
from sys import exit
from datetime import time

pd.set_option('display.max_columns', None)

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.notifica import Notifica
from bulletin.utils.normalize import normalize_hash, normalize_labels

In [4]:
read_cc = True and False
download_nt = True and False
read_nt = True and False

In [5]:
notifica = Notifica()
if download_nt:
    notifica.download_todas_notificacoes()
    notifica.read_todas_notificacoes()
elif read_nt:
    notifica.read_todas_notificacoes()
else:
    notifica.load()

casosn = notifica.get_casos()
print(f"casosn len: {len(casosn)}\n")
print(casosn.columns.values)
casosn.groupby('classificacao_final')[['id']].count()

casosn len: 3076573

['id' 'cns' 'paciente' 'data_nascimento' 'nome_mae' 'cpf' 'sexo'
 'uf_residencia' 'ibge_residencia' 'classificacao_final'
 'criterio_classificacao' 'evolucao' 'data_1o_sintomas' 'data_cura_obito'
 'metodo' 'exame' 'resultado' 'data_coleta' 'data_recebimento'
 'data_liberacao' 'status_notificacao' 'excluir_ficha' 'origem'
 'uf_unidade_notifica' 'ibge_unidade_notifica' 'data_notificacao'
 'updated_at' 'data_diagnostico' 'idade' 'mun_resid' 'uf_resid' 'rs'
 'mun_atend' 'uf_atend' 'hash_resid' 'hash_atend' 'hash_mae' 'hash_nasc'
 'hash_diag' 'hash_obito' 'checksum']


,id
classificacao_final,
0,617
1,1048576
2,971836
3,1048576
5,6968


In [6]:
cc = CasosConfirmados()
if read_cc:
    cc.read()
    cc.save()
else:
    cc.load()

casosc = cc.get_casos()
print(f"casosc len: {len(casosc)}\n")
print(casosc.columns.values)

loading Casos Confirmados time elepsed:00:00:02
casosc len: 953884

['ordem' 'identificacao' 'ibge_res_pr' 'rs_res_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'rs' 'laboratorio' 'dt_diag' 'comunicacao' 'is'
 'obito' 'data_obito' 'excluir' 'hash' 'hash_less' 'hash_more'
 'hash_atend' 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [7]:
notificacoes_encontradas = casosn.loc[
    (casosn['classificacao_final'] == 2) &
    (casosn['excluir_ficha'] == 2) &
    casosn['status_notificacao'].isin([1,2]) &
    casosn['hash_resid'].isin(np.concatenate((casosc['hash'].values,casosc['hash_less'].values,casosc['hash_more'].values))) |
    casosn['hash_atend'].isin(np.concatenate((casosc['hash_atend'].values,casosc['hash_less_atend'].values,casosc['hash_more_atend'].values))) |
    casosn['hash_diag'].isin(casosc['hash_diag']) |
    casosn['hash_obito'].isin(casosc['hash_obito'])
]

print(len(notificacoes_encontradas))
notificacoes_encontradas.groupby('classificacao_final')[['id']].count()

1054416


,id
classificacao_final,
0,17
1,58199
2,935787
3,59847
5,566


In [8]:
#hash obito
casosnc = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash_obito'].isin(notificacoes_encontradas['hash_obito']))],notificacoes_encontradas[['hash_obito','id']], how='left', on='hash_obito')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

22107
933260


In [9]:
#hash_diag
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_diag'].isin(notificacoes_encontradas['hash_diag'])],notificacoes_encontradas[['hash_diag','id']], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

875179
97895


In [10]:
#hash
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash'].isin(notificacoes_encontradas['hash_resid'])],notificacoes_encontradas[['hash_resid','id']], how='left',
left_on='hash',right_on='hash_resid'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

936888
43778


In [11]:
#hash_less
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_less'].isin(notificacoes_encontradas['hash_resid'])],notificacoes_encontradas[['hash_resid','id']], how='left',
left_on='hash_less',right_on='hash_resid'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

937778
42958


In [12]:
#hash_more
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_more'].isin(notificacoes_encontradas['hash_resid'])],notificacoes_encontradas[['hash_resid','id']], how='left',
left_on='hash_more',right_on='hash_resid'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

938371
42419


In [13]:
#hash_atend
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_atend'].isin(notificacoes_encontradas['hash_atend'])],notificacoes_encontradas[['hash_atend','id']], how='left',
left_on='hash_atend',right_on='hash_atend'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

943095
38130


In [14]:
#hash_less_atend
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_less_atend'].isin(notificacoes_encontradas['hash_atend'])],notificacoes_encontradas[['hash_atend','id']], how='left',
left_on='hash_less_atend',right_on='hash_atend'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

943196
38040


In [15]:
#hash_more_atend
casosnc =  casosnc.append(pd.merge(casosc.loc[casosc['hash_more_atend'].isin(notificacoes_encontradas['hash_atend'])],notificacoes_encontradas[['hash_atend','id']], how='left',
left_on='hash_more_atend',right_on='hash_atend'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosnc['ordem'])].index)
print(len(casosnc))
print(len(casosc))

943279
37966


In [16]:
duplicados = casosnc.loc[casosnc.duplicated('ordem', keep=False)]
casosnc = casosnc.loc[~casosnc['ordem'].isin(duplicados['ordem'])]

In [17]:
duplicados

,ordem,identificacao,ibge_res_pr,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito,id,hash_resid,hash_atend_x,hash_atend_y
0,109,10853,410940,5,FERNANDO JOSE COSTA,M,57,GUARAPUAVA,GUARAPUAVA,5,LACEN,2020-03-26,2020-03-27,2020-03-08,SIM,2020-06-15,NaN,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA26032020,FERNANDOJOSECOSTA15062020,83233,NaN,NaN,NaN
1,109,10853,410940,5,FERNANDO JOSE COSTA,M,57,GUARAPUAVA,GUARAPUAVA,5,LACEN,2020-03-26,2020-03-27,2020-03-08,SIM,2020-06-15,NaN,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA26032020,FERNANDOJOSECOSTA15062020,90860,NaN,NaN,NaN
28,944,29260,410690,2,SIMAO SAPORITI DE SIQUEIRA,M,76,CURITIBA,CURITIBA,2,GENOPRIMER,2020-04-17,2020-04-18,NaT,SIM,2020-06-30,NaN,SIMAOSAPORITIDESIQUEIRA76CURITIBA,SIMAOSAPORITIDESIQUEIRA75CURITIBA,SIMAOSAPORITIDESIQUEIRA77CURITIBA,SIMAOSAPORITIDESIQUEIRA76CURITIBA,SIMAOSAPORITIDESIQUEIRA75CURITIBA,SIMAOSAPORITIDESIQUEIRA77CURITIBA,SIMAOSAPORITIDESIQUEIRA17042020,SIMAOSAPORITIDESIQUEIRA30062020,114234,NaN,NaN,NaN
29,944,29260,410690,2,SIMAO SAPORITI DE SIQUEIRA,M,76,CURITIBA,CURITIBA,2,GENOPRIMER,2020-04-17,2020-04-18,NaT,SIM,2020-06-30,NaN,SIMAOSAPORITIDESIQUEIRA76CURITIBA,SIMAOSAPORITIDESIQUEIRA75CURITIBA,SIMAOSAPORITIDESIQUEIRA77CURITIBA,SIMAOSAPORITIDESIQUEIRA76CURITIBA,SIMAOSAPORITIDESIQUEIRA75CURITIBA,SIMAOSAPORITIDESIQUEIRA77CURITIBA,SIMAOSAPORITIDESIQUEIRA17042020,SIMAOSAPORITIDESIQUEIRA30062020,198323,NaN,NaN,NaN
38,1149,23732,410765,2,MILTON LUIZ CIAPPINA,M,71,FAZENDA RIO GRANDE,CURITIBA,2,GENOPRIMER,2020-04-25,2020-04-26,NaT,SIM,2020-05-03,NaN,MILTONLUIZCIAPPINA71FAZENDARIOGRANDE,MILTONLUIZCIAPPINA70FAZENDARIOGRANDE,MILTONLUIZCIAPPINA72FAZENDARIOGRANDE,MILTONLUIZCIAPPINA71CURITIBA,MILTONLUIZCIAPPINA70CURITIBA,MILTONLUIZCIAPPINA72CURITIBA,MILTONLUIZCIAPPINA25042020,MILTONLUIZCIAPPINA03052020,67234,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943245,102365,158385,410180,2,JESSICA APARECIDA DOS SANTOS,F,28,ARAUCARIA,CURITIBA,2,UNIMED,2020-08-14,2020-08-15,NaT,NAO,NaT,NaN,JESSICAAPARECIDADOSSANTOS28ARAUCARIA,JESSICAAPARECIDADOSSANTOS27ARAUCARIA,JESSICAAPARECIDADOSSANTOS29ARAUCARIA,NaN,JESSICAAPARECIDADOSSANTOS27CURITIBA,JESSICAAPARECIDADOSSANTOS29CURITIBA,JESSICAAPARECIDADOSSANTOS14082020,NaN,1201031,NaN,JESSICAAPARECIDADOSSANTOS28CURITIBA,JESSICAAPARECIDADOSSANTOS29CURITIBA
943260,152805,210953,999999,99,ELTON DA SILVA NUNES,M,31,MARITUBA/PA,CURITIBA,99,IBMP,2020-09-12,2020-09-13,2020-09-09,NAO,NaT,NaN,ELTONDASILVANUNES31MARITUBAPA,ELTONDASILVANUNES30MARITUBAPA,ELTONDASILVANUNES32MARITUBAPA,NaN,ELTONDASILVANUNES30CURITIBA,ELTONDASILVANUNES32CURITIBA,ELTONDASILVANUNES12092020,NaN,1350043,NaN,ELTONDASILVANUNES31CURITIBA,ELTONDASILVANUNES32CURITIBA
943261,152805,210953,999999,99,ELTON DA SILVA NUNES,M,31,MARITUBA/PA,CURITIBA,99,IBMP,2020-09-12,2020-09-13,2020-09-09,NAO,NaT,NaN,ELTONDASILVANUNES31MARITUBAPA,ELTONDASILVANUNES30MARITUBAPA,ELTONDASILVANUNES32MARITUBAPA,NaN,ELTONDASILVANUNES30CURITIBA,ELTONDASILVANUNES32CURITIBA,ELTONDASILVANUNES12092020,NaN,1352641,NaN,ELTONDASILVANUNES31CURITIBA,ELTONDASILVANUNES32CURITIBA
943269,228331,305910,410150,16,FABIANA SALUM LIBOS,F,48,ARAPONGAS,LONDRINA,16,MAGLUMI IGM DE 2019-NCOV,2020-07-25,2020-11-13,NaT,NAO,NaT,NaN,FABIANASALUMLIBOS48ARAPONGAS,FABIANASALUMLIBOS47ARAPONGAS,FABIANASALUMLIBOS49ARAPONGAS,NaN,FABIANASALUMLIBOS47LONDRINA,FABIANASALUMLIBOS49LONDRINA,FABIANASALUMLIBOS25072020,NaN,215721,NaN,FABIANASALUMLIBOS48LONDRINA,FABIANASALUMLIBOS49LONDRINA
